<a href="https://colab.research.google.com/github/prog815/learnTensorFlow/blob/master/tf_data_%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B2%D1%85%D0%BE%D0%B4%D0%BD%D1%8B%D1%85_%D0%BA%D0%BE%D0%BD%D0%B2%D0%B5%D0%B9%D0%B5%D1%80%D0%BE%D0%B2_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tf.data: создание входных конвейеров TensorFlow.

https://www.tensorflow.org/guide/data

Енин А.В. 

16.08.2020

Оренбург

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,4,3,2,1])
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [ ]:
for elem in dataset:
    print(elem.numpy())

1
2
3
4
5
4
3
2
1


In [ ]:
it = iter(dataset)
print(next(it).numpy())

1


In [ ]:
dataset.reduce(0,lambda st,vl : st + vl**2).numpy()

85

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform((4,3)))
dataset1.element_spec

TensorSpec(shape=(3,), dtype=tf.float32, name=None)

In [ ]:
it = iter(dataset1)
it.next()

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.3890667, 0.2886604, 0.3759637], dtype=float32)>

In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices((
    tf.random.uniform((4,)),
    tf.random.uniform((4,3),maxval=10,dtype=tf.int32)
))
dataset2.element_spec

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(3,), dtype=tf.int32, name=None))

In [ ]:
d = dataset2.take(1)
print(d)
d.as_numpy_iterator().next()

<TakeDataset shapes: ((), (3,)), types: (tf.float32, tf.int32)>


(0.40140188, array([8, 1, 0], dtype=int32))

In [ ]:
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))
dataset3.element_spec

(TensorSpec(shape=(3,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(3,), dtype=tf.int32, name=None)))

In [ ]:
t = tf.SparseTensor(indices=((0,0),(1,2)),values=(1,2),dense_shape=(3,4))
dataset4 = tf.data.Dataset.from_tensors(t)
dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [ ]:
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

# Преобразования

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform((4,10),minval=1,maxval=10,dtype=tf.int32)
)
dataset1

<TensorSliceDataset shapes: (10,), types: tf.int32>

In [ ]:
for z in dataset1:
    print(z)

tf.Tensor([9 6 8 9 4 8 6 5 8 9], shape=(10,), dtype=int32)
tf.Tensor([8 3 3 2 8 3 6 4 3 2], shape=(10,), dtype=int32)
tf.Tensor([4 3 9 5 4 3 4 6 4 2], shape=(10,), dtype=int32)
tf.Tensor([6 2 3 2 5 6 9 5 9 1], shape=(10,), dtype=int32)


In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (
        tf.random.uniform((4,)),
        tf.random.uniform((4,100),maxval=10,dtype=tf.int32)
    )
)
dataset2

<TensorSliceDataset shapes: ((), (100,)), types: (tf.float32, tf.int32)>

In [ ]:
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))
dataset3

<ZipDataset shapes: ((10,), ((), (100,))), types: (tf.int32, (tf.float32, tf.int32))>

In [ ]:
for a,(b,c) in dataset3:
    print('shapes:',a.shape,b.shape,c.shape)

shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)


# Чтение входных данных

In [ ]:
train,test = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train[0].shape,train[1].shape

((60000, 28, 28), (60000,))

In [ ]:
images,labels = train
images = images / 255

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((images,labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

## Использование генераторов

In [ ]:
def count(stop):
    i = 0
    while i<stop:
        yield i
        i += 1

In [ ]:
for n in count(5):
    print(n)

0
1
2
3
4


In [ ]:
ds_counter = tf.data.Dataset.from_generator(count,output_types=tf.int32,output_shapes=(),args=[25])
ds_counter

<FlatMapDataset shapes: (), types: tf.int32>

In [ ]:
for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]


In [ ]:
import numpy as np

np.set_printoptions(precision=4)

def gen_series():
    i = 0
    while True:
        size = np.random.randint(1,10)
        yield i, np.random.randn(size)
        i += 1

In [ ]:
for i,series in gen_series():
    print(i,':',str(series))
    if i>5:
        break

0 : [-0.0963 -1.5694 -1.945  -1.4031 -0.1682]
1 : [-0.2191  1.6038 -0.2073  0.036   2.5549]
2 : [-0.7925  0.4341  0.1008  1.3615 -1.519   0.4369]
3 : [-1.4789  1.5803 -1.292  -0.8222 -0.2652  0.621  -0.8616 -2.1116]
4 : [-0.1103]
5 : [ 0.6304 -1.125 ]
6 : [-0.3275 -1.5769 -0.1282]


In [ ]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types = (tf.int32,tf.float32),
    output_shapes = ((),(None,))
)

print(ds_series)

ds_series_batch = ds_series.shuffle(20)
for item in ds_series_batch.take(3):
    print(item)

<FlatMapDataset shapes: ((), (None,)), types: (tf.int32, tf.float32)>
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.1749,  1.0353], dtype=float32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=14>, <tf.Tensor: shape=(8,), dtype=float32, numpy=
array([-0.1767,  0.664 ,  0.0115, -0.6613,  0.3888, -1.3676,  0.8567,
       -0.5775], dtype=float32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=13>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.1637, -0.7205,  0.6247], dtype=float32)>)


In [ ]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types = (tf.int32,tf.float32),
    output_shapes = ((),(None,))
)

print(ds_series)

ds_series_batch = ds_series.shuffle(20).padded_batch(4)
for ids,sequence_batch in ds_series_batch.take(1):
    print(ids.numpy())
    print()
    print(sequence_batch.numpy())

<FlatMapDataset shapes: ((), (None,)), types: (tf.int32, tf.float32)>
[ 5  7 17 19]

[[ 0.8057 -1.1057  0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.9916 -0.6004 -1.1232  0.      0.      0.      0.      0.      0.    ]
 [ 0.4303 -1.0222 -0.9614  0.6349 -1.3889 -2.0788 -0.0897  0.3258  2.0922]
 [ 0.0584  0.      0.      0.      0.      0.      0.      0.      0.    ]]


In [ ]:
flowers = tf.keras.utils.get_file('flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',untar=True)

228818944/228813984 [==============================] - 2s 0us/step


In [ ]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=20)

In [ ]:
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [ ]:
print(images.dtype,images.shape)
print(labels.dtype,labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 5)


In [ ]:
 ds = tf.data.Dataset.from_generator(
    img_gen.flow_from_directory, args=[flowers], 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,256,256,3], [32,5])
)

ds
 

<FlatMapDataset shapes: ((32, 256, 256, 3), (32, 5)), types: (tf.float32, tf.float32)>

## Использование данных TFRecord

In [ ]:
 # Creates a dataset that reads all of the examples from two files.
fsns_test_file = tf.keras.utils.get_file("fsns.tfrec", "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")
 

7905280/7904079 [==============================] - 0s 0us/step


In [ ]:
dataset = tf.data.TFRecordDataset(filenames=[fsns_test_file])
dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

## Использование текстовых данных 

In [ ]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']
file_paths = [
    tf.keras.utils.get_file(file_name, directory_url + file_name)
    for file_name in file_names
]

811008/807992 [==============================] - 0s 0us/step


In [ ]:
file_paths

['/root/.keras/datasets/cowper.txt',
 '/root/.keras/datasets/derby.txt',
 '/root/.keras/datasets/butler.txt']

In [ ]:
! head /root/.keras/datasets/cowper.txt

﻿Achilles sing, O Goddess! Peleus' son;
His wrath pernicious, who ten thousand woes
Caused to Achaia's host, sent many a soul
Illustrious into Ades premature,
And Heroes gave (so stood the will of Jove)
To dogs and to all ravening fowls a prey,
When fierce dispute had separated once
The noble Chief Achilles from the son
Of Atreus, Agamemnon, King of men.
Who them to strife impell'd? What power divine?


In [ ]:
dataset = tf.data.TextLineDataset(file_paths)
dataset

<TextLineDatasetV2 shapes: (), types: tf.string>

In [ ]:
for line in dataset.take(5):
    print(line.numpy())

b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b'His wrath pernicious, who ten thousand woes'
b"Caused to Achaia's host, sent many a soul"
b'Illustrious into Ades premature,'
b'And Heroes gave (so stood the will of Jove)'


## Использование данных CSV

In [ ]:
 titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

32768/30874 [===============================] - 0s 0us/step
